In [1]:
"""Shivam Giri
cdac silchar
asssignment 2 """
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=0d530e8d7cef20ad1a431afe8fc6c7346f856d3407e85b5d50886bfed404311a
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

In [4]:
# create the context
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.3
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 1.8.0_422
Branch HEAD
Compiled by user haejoon.lee on 2024-09-09T05:20:05Z
Revision 32232e9ed33bb16b93ad58cfde8b82e0f07c0970
Url https://github.com/apache/spark
Type --help for more information.


In [6]:
from google.colab import files
files.upload()

Saving people.json to people.json


{'people.json': b'{"name":"Michael"}\n{"name":"Andy", "age":30}\n{"name":"Justin", "age":19}\n{"name":"Mary", "age":29}\n{"name":"John"}\n{"name":"Juhi", "age":36}\n{"name":"Jasmeen", "age":13}\n{"name":"Seeta", "age":12}\n{"age":45}\n{"name":"Mohan", "age":34}\n{"name":"Kayen", "age":18}\n{"age":67}\n{"name":"Caley", "age":21}\n{"name":"Shyam", "age":19}\n'}

In [7]:
df = spark.read.json("people.json")

In [8]:
df.show(10)

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
|  29|   Mary|
|NULL|   John|
|  36|   Juhi|
|  13|Jasmeen|
|  12|  Seeta|
|  45|   NULL|
|  34|  Mohan|
+----+-------+
only showing top 10 rows



In [9]:
#Count number of records
print("The total number of rows in df is : ",df.count())


The total number of rows in df is :  14


In [10]:
#2. create a new column with the simple copy of age column.
from pyspark.sql.functions import *
df = df.withColumn("age2",col("age"))
df.show()

+----+-------+----+
| age|   name|age2|
+----+-------+----+
|NULL|Michael|NULL|
|  30|   Andy|  30|
|  19| Justin|  19|
|  29|   Mary|  29|
|NULL|   John|NULL|
|  36|   Juhi|  36|
|  13|Jasmeen|  13|
|  12|  Seeta|  12|
|  45|   NULL|  45|
|  34|  Mohan|  34|
|  18|  Kayen|  18|
|  67|   NULL|  67|
|  21|  Caley|  21|
|  19|  Shyam|  19|
+----+-------+----+



In [11]:

#3. create a new column containing ages of people one year bigger.

df = df.withColumn("age1y",col("age")+1)
df.show()

+----+-------+----+-----+
| age|   name|age2|age1y|
+----+-------+----+-----+
|NULL|Michael|NULL| NULL|
|  30|   Andy|  30|   31|
|  19| Justin|  19|   20|
|  29|   Mary|  29|   30|
|NULL|   John|NULL| NULL|
|  36|   Juhi|  36|   37|
|  13|Jasmeen|  13|   14|
|  12|  Seeta|  12|   13|
|  45|   NULL|  45|   46|
|  34|  Mohan|  34|   35|
|  18|  Kayen|  18|   19|
|  67|   NULL|  67|   68|
|  21|  Caley|  21|   22|
|  19|  Shyam|  19|   20|
+----+-------+----+-----+



In [12]:
#4. Show the records where age column is missing.
df.filter(col("age").isNull()).show()

+----+-------+----+-----+
| age|   name|age2|age1y|
+----+-------+----+-----+
|NULL|Michael|NULL| NULL|
|NULL|   John|NULL| NULL|
+----+-------+----+-----+



In [13]:
#6. Fill the missing values of age column with mean.
mean_sal = df.select(mean(df['age'])).collect()[0][0]
print("The mean salary is : ", mean_sal)

The mean salary is :  28.583333333333332


In [14]:
df1 = df.na.fill(mean_sal,subset=['age'])
df1.show()

+---+-------+----+-----+
|age|   name|age2|age1y|
+---+-------+----+-----+
| 28|Michael|NULL| NULL|
| 30|   Andy|  30|   31|
| 19| Justin|  19|   20|
| 29|   Mary|  29|   30|
| 28|   John|NULL| NULL|
| 36|   Juhi|  36|   37|
| 13|Jasmeen|  13|   14|
| 12|  Seeta|  12|   13|
| 45|   NULL|  45|   46|
| 34|  Mohan|  34|   35|
| 18|  Kayen|  18|   19|
| 67|   NULL|  67|   68|
| 21|  Caley|  21|   22|
| 19|  Shyam|  19|   20|
+---+-------+----+-----+



In [15]:
#7. Write a sql query to find the people with age greater than 19.
df.filter(col("age")>19).show()

+---+-----+----+-----+
|age| name|age2|age1y|
+---+-----+----+-----+
| 30| Andy|  30|   31|
| 29| Mary|  29|   30|
| 36| Juhi|  36|   37|
| 45| NULL|  45|   46|
| 34|Mohan|  34|   35|
| 67| NULL|  67|   68|
| 21|Caley|  21|   22|
+---+-----+----+-----+

